# Класс Research: эксперименты с пайплайнами
Предназначен для многократного запуска одних и тех же пайплайнов с различными параметрами. Рассмотрим простой пример: запуск пайплайнов vgg7_train и vgg7_test на MNIST с различными значениями layout.

### Пример использования

In [1]:
import sys

sys.path.append('..')

from dataset import B, C, V, Config
from dataset.opensets import MNIST
from dataset.models.tf import VGG7

In [2]:
model_config = {
    'inputs/images': {
        'shape': (28, 28, 1),
        'type': 'float32',
        'name': 'reshaped_images'
    },
    'inputs/labels': {
        'classes': 10,
        'type': 'int32',
        'transform': 'ohe',
        'name': 'targets'
    },
    'input_block/inputs': 'images',
    'output/ops': ['accuracy'],
    'loss': 'ce',
    'optimizer': 'Adam'
}

feed_dict = {'images': B('images'),
             'labels': B('labels')}

In [3]:
pipeline_config = Config(model_config=model_config)

In [4]:
mnist = MNIST()

vgg7_train = (mnist.train.p
              .init_variable('loss', init_on_each_run=list)
              .init_model('dynamic', VGG7, 'model', C('model_config'))
              .train_model('model', feed_dict=feed_dict, fetches='loss', save_to=V('loss'), mode='a')
              .run(batch_size=8, shuffle=True, n_epochs=None, lazy=True)
             )

vgg7_test = (mnist.test.p
             .init_variable('accuracy', init_on_each_run=list)
             .import_model('model', C('import_model_from'))
             .predict_model('model', feed_dict=feed_dict, fetches='output_accuracy', save_to=V('accuracy'), mode='a')
             .run(batch_size=10, shuffle=True, n_epochs=None, lazy=True)
            )

Extracting C:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gzExtractingExtractingExtracting
   C:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gz




In [5]:
grid_config = {'model_config/body/block/layout': ['cna', 'can', 'acn']}

In [6]:
from research import Research

mr = Research()
mr.add_pipeline(vgg7_train, 'loss', config=pipeline_config, name='train')
mr.add_pipeline(vgg7_test, 'accuracy', config=pipeline_config, import_model_from='train')
mr.add_grid_config(grid_config)
mr.run(n_reps=2, n_iters=100, n_jobs=1, name='my_research')

Далее мы более подробно разберём параметры метода run и требования к пайплайнам. Результаты экспериментов будут лежать в папке 'my_research/results'. Для каждой конфигурации будет создана отдельная папка, в которой будут лежать n_reps файлов (если в ходе эксперимента ничего не поломалось). Каждый файл - это pickle.dump со значениями переменных после указанного числа итераций. Названия папок будут следующие:

Плохо, что '/' будет командой по созданию подпапок. Можно было бы просто заменить его на какой-то другой символ, но кроме того, сами значения параметров могут быть более сложными. Например, grid может выглядеть следующим образом:

In [7]:
grid = {
    'model_config/head/units': [[100, 100, 10], [100, 120, 10], [4096, 4096, 10]],
    'model_config/body/block/layout': ['cna', 'can', 'acn']
}

Тогда строчные представления элементов сетки параметров будут иметь вид

Слишком сложная структура, хотя параметров всего 2. Избежать этого помогут классы KV, Option и Grid.

### Классы KV, Option и Grid
Введены для удобной работы с сетками параметров, из которых будут генерироваться конфигурации пайплайнов для экспериментов.

In [8]:
from grid import KV, Option, Grid

KV (key: value) - это просто пара значение-псевдоним. Добавим псевдонимы к названиям параметров:

In [9]:
p1 = KV('model_config/body/block/layout', 'layout')
p2 = KV('model_config/head/units', 'head')

In [10]:
p1.value, p1.alias

('model_config/body/block/layout', 'layout')

Также добавим псевдонимы к значениям 'model_config/head/units':

In [11]:
v1 = KV([100, 100, 10], 'small')
v2 = KV([4096, 4096, 10], 'big')

Option - это пара параметр-список значений, которые хотим перебрать. Как параметр, так и значения могут быть представлены как эеземпляры KV.

In [12]:
op1 = Option(p1, ['cna', 'acn', 'can'])
op2 = Option(p2, [v1, v2])

In [13]:
op1.option(), op1.alias()

({'model_config/body/block/layout': ['cna', 'acn', 'can']},
 {'layout': ['cna', 'acn', 'can']})

In [14]:
op2.option(), op2.alias()

({'model_config/head/units': [[100, 100, 10], [4096, 4096, 10]]},
 {'head': ['small', 'big']})

Экземпляры Option можно складывать и перемножать. Результатом будет экземпляр класса Grid.

In [15]:
op1 * op2

Grid([[{'layout': ['cna', 'acn', 'can']}, {'head': ['small', 'big']}]])

In [16]:
op1 + op2

Grid([[{'layout': ['cna', 'acn', 'can']}], [{'head': ['small', 'big']}]])

В начальном примере сетка просто создавалась из словаря:

In [17]:
grid = {
    'model_config/head/units': [[100, 100, 10], [100, 120, 10], [4096, 4096, 10]],
    'model_config/body/block/layout': ['cna', 'can', 'acn']
}

Grid(grid)

Grid([[{'model_config/body/block/layout': ['cna', 'can', 'acn']}, {'model_config/head/units': [[100, 100, 10], [100, 120, 10], [4096, 4096, 10]]}]])

На самом деле, Grid представляет собой следующую структуру:

$[[op_{11}, op_{12}, \dots, op_{1n_1}] \\
 \dots \\
 [op_{m1}, op_{m2}, \dots, op_{mn_m}]]$ 

При генерировании конфигураций опции внутри каждого списка декартово перемножаются, затем конфигурации всех списков конкатенируются. Это проще понять на примере:

In [18]:
list((op1 * op2).gen_configs())

[ConfigAlias({'head': 'small', 'layout': 'cna'}),
 ConfigAlias({'head': 'big', 'layout': 'cna'}),
 ConfigAlias({'head': 'small', 'layout': 'acn'}),
 ConfigAlias({'head': 'big', 'layout': 'acn'}),
 ConfigAlias({'head': 'small', 'layout': 'can'}),
 ConfigAlias({'head': 'big', 'layout': 'can'})]

In [19]:
list((op1 + op2).gen_configs())

[ConfigAlias({'layout': 'cna'}),
 ConfigAlias({'layout': 'acn'}),
 ConfigAlias({'layout': 'can'}),
 ConfigAlias({'head': 'small'}),
 ConfigAlias({'head': 'big'})]

Метод gen_configs класса Grid возвращает генератор конфигураций. Точнее, генератор создаёт объекты класса ConfigAlias. У этих объектов есть значение (объект класса Config), и псевдоним.

In [20]:
op3 = Option('new_parameter', ['value1', 'value2'])

In [21]:
grid = (op1 + op2) * op3
[item.config().flatten() for item in grid.gen_configs()]

[{'model_config/body/block/layout': 'cna', 'new_parameter': 'value1'},
 {'model_config/body/block/layout': 'cna', 'new_parameter': 'value2'},
 {'model_config/body/block/layout': 'acn', 'new_parameter': 'value1'},
 {'model_config/body/block/layout': 'acn', 'new_parameter': 'value2'},
 {'model_config/body/block/layout': 'can', 'new_parameter': 'value1'},
 {'model_config/body/block/layout': 'can', 'new_parameter': 'value2'},
 {'model_config/head/units': [100, 100, 10], 'new_parameter': 'value1'},
 {'model_config/head/units': [100, 100, 10], 'new_parameter': 'value2'},
 {'model_config/head/units': [4096, 4096, 10], 'new_parameter': 'value1'},
 {'model_config/head/units': [4096, 4096, 10], 'new_parameter': 'value2'}]

In [22]:
[item.alias() for item in grid.gen_configs()]

[{'layout': 'cna', 'new_parameter': 'value1'},
 {'layout': 'cna', 'new_parameter': 'value2'},
 {'layout': 'acn', 'new_parameter': 'value1'},
 {'layout': 'acn', 'new_parameter': 'value2'},
 {'layout': 'can', 'new_parameter': 'value1'},
 {'layout': 'can', 'new_parameter': 'value2'},
 {'head': 'small', 'new_parameter': 'value1'},
 {'head': 'small', 'new_parameter': 'value2'},
 {'head': 'big', 'new_parameter': 'value1'},
 {'head': 'big', 'new_parameter': 'value2'}]

Теперь, когда у конфигов есть псевдонимы, мы можем получить удобное представление для конфигурций:

In [23]:
[item.alias(as_string=True) for item in grid.gen_configs()]

['layout-cna_new_parameter-value1',
 'layout-cna_new_parameter-value2',
 'layout-acn_new_parameter-value1',
 'layout-acn_new_parameter-value2',
 'layout-can_new_parameter-value1',
 'layout-can_new_parameter-value2',
 'head-small_new_parameter-value1',
 'head-small_new_parameter-value2',
 'head-big_new_parameter-value1',
 'head-big_new_parameter-value2']

## Подробнее о Research